In [ ]:
import socket
import threading

class OrderBook:
    def __init__(self):
        self.buy_levels = []
        self.sell_levels = []

    def add_order(self, side, price, quantity):
        levels = self.buy_levels if side == 'buy' else self.sell_levels
        opposite_levels = self.sell_levels if side == 'buy' else self.buy_levels

        trade_messages, remaining_quantity = self.match_orders(opposite_levels, side, price, quantity)
        if remaining_quantity > 0:
            self.add_to_levels(levels, price, remaining_quantity, side)
            levels.sort(key=lambda x: x['price'], reverse=(side == 'buy'))

        return trade_messages

    def add_to_levels(self, levels, price, quantity, side):
        for level in levels:
            if level['price'] == price:
                level['quantity'] += quantity
                return
        new_level = {'price': price, 'quantity': quantity}
        inserted = False
        for i, level in enumerate(levels):
            if (side == 'buy' and price > level['price']) or (side == 'sell' and price < level['price']):
                levels.insert(i, new_level)
                inserted = True
                break
        if not inserted:
            levels.append(new_level)

    def match_orders(self, opposite_levels, side, price, quantity):
        trade_messages = []
        i = 0
        while i < len(opposite_levels) and quantity > 0:
            level = opposite_levels[i]
            if (side == 'buy' and price >= level['price']) or (side == 'sell' and price <= level['price']):
                if quantity >= level['quantity']:
                    trade_messages.append(f"Trade executed: {level['quantity']} at {level['price']}")
                    quantity -= level['quantity']
                    opposite_levels.pop(i)
                else:
                    trade_messages.append(f"Trade executed: {quantity} at {level['price']}")
                    level['quantity'] -= quantity
                    quantity = 0
            else:
                break
        return trade_messages, quantity

    def print_levels(self):
        max_length = max(len(self.buy_levels), len(self.sell_levels))
        print(f"{'Buy Price':<15} {'Buy Quantity':<15} {'Sell Price':<15} {'Sell Quantity':<15}")
        print("-" * 60)
        for i in range(max_length):
            buy_level = self.buy_levels[i] if i < len(self.buy_levels) else {'price': '', 'quantity': ''}
            sell_level = self.sell_levels[i] if i < len(self.sell_levels) else {'price': '', 'quantity': ''}
            print(f"{buy_level['price']:<15} {buy_level['quantity']:<15} {sell_level['price']:<15} {sell_level['quantity']:<15}")

def handle_client(conn, addr, order_book):
    print(f"Connected by {addr}")
    while True:
        data = conn.recv(4096)
        if not data:
            break
        parts = data.decode().split(',')
        if len(parts) == 4:
            asset_id, side, price, quantity = parts
            trade_messages = order_book.add_order(side, int(price), int(quantity))
            order_book.print_levels()
            for message in trade_messages:
                conn.sendall(message.encode() + b"\n")
            conn.sendall(b"Order received and processed\n")
        else:
            conn.sendall(b"Invalid order format\n")
    conn.close()

def start_server():
    order_book = OrderBook()
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    port = 56789
    server.bind(('127.0.0.1', port))
    server.listen(10)
    print(f"Server started and listening on port {port}")
    while True:
        client_socket, addr = server.accept()
        client_thread = threading.Thread(target=handle_client, args=(client_socket, addr, order_book))
        client_thread.start()

start_server()


Server started and listening on port 56789
